In [1]:
import argparse
import sys
import numpy as np
from math import ceil
from time import time
from itertools import product
from scipy.stats import truncnorm
import matplotlib.pyplot as plt

try:
    from img_to_mat.img_to_mat import img_to_mat, mat_to_img
except ImportError:
    print('run the following from the img_to_mat directory and try again:')
    print('python setup.py build_ext --inplace')

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
# Part 2
# Weight and bias

def weight_variable(shape):
    """
    Initialize weight matrices with truncated normal distribution with standard deviation of 0.1.

    In general, shape = [filter_height, filter_width, in_channels, out_channels].
    """
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init)

def bias_variable(shape):
    """
    Initialize bias vectors with constant value of 0.1.

    In general, shape = [out_channels].
    """
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)


# Layers

def conv2d(x, W):
    """2D convolution of x with filter W, unit stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
    """2x2 max pooling layer with non-overlapping kernel stride."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def flatten(x):
    size = 1
    for dim in x.shape.as_list()[1:]:
        size *= dim
    return tf.reshape(x, [-1, size])

def fullconn(x, W):
    return tf.matmul(x, W)

In [3]:
class NN:
    class Utils:
        def pad(x, ksize, strides, padding='SAME'):
            batch_size, in_height, in_width, in_channels = x.shape
            k_batch_size, k_height, k_width, k_channels = ksize
            if padding == 'SAME':
                out_height = ceil(float(in_height) / float(strides[1]))
                out_width = ceil(float(in_width) / float(strides[2]))
                if (in_height % strides[1] == 0):
                    pad_height = max(k_height - strides[1], 0)
                else:
                    pad_height = max(k_height - (in_height % strides[1]), 0)
                if (in_width % strides[2] == 0):
                    pad_width = max(k_width - strides[2], 0)
                else:
                    pad_width = max(k_width - (in_width % strides[2]), 0)
                pad_top = pad_height // 2
                pad_bottom = pad_height - pad_top
                pad_left = pad_width // 2
                pad_right = pad_width - pad_left
            elif padding == 'VALID':
                out_height = ceil(float(in_height - k_height + 1) / float(strides[1]))
                out_width = ceil(float(in_width - k_width + 1) / float(strides[2]))
                pad_top = 0
                pad_bottom = 0
                pad_left = 0
                pad_right = 0
            return pad_top, pad_bottom, pad_left, pad_right, out_height, out_width
        
        # def img_to_col(x, out_height, out_width, filter_height, filter_width, strides):
        #     batch_size = x.shape[0]
        #     in_channels = x.shape[3]
        #     x_col = np.zeros((batch_size * out_height * out_width, filter_height * filter_width * in_channels))
        #     for b in range(batch_size):
        #         for i in range(out_height):
        #             for j in range(out_width):
        #                 row = b * out_height * out_width + i * out_width + j
        #                 for p in range(filter_height):
        #                     for q in range(filter_width):
        #                         for r in range(in_channels):
        #                             col = p * filter_width * in_channels + q * in_channels + r
        #                             x_col[row, col] += x[b, strides[1] * i + p, strides[2] * j + q, r]
        #     return x_col
        
        # def col_to_img(x_col, out_height, out_width, in_height, in_width, filter_height, filter_width, strides):
        #     batch_size = x_col.shape[0] // (out_height * out_width)
        #     in_channels = x_col.shape[1] // (filter_height * filter_width)
        #     x = np.zeros((batch_size, in_height, in_width, in_channels))
        #     for b in range(batch_size):
        #         for i in range(out_height):
        #             for j in range(out_width):
        #                 row = b * out_height * out_width + i * out_width + j
        #                 for p in range(filter_height):
        #                     for q in range(filter_width):
        #                         for r in range(in_channels):
        #                             col = p * filter_width * in_channels + q * in_channels + r
        #                             x[b, strides[1] * i + p, strides[2] * j + q, r] += x_col[row, col]
        #     return x
        
        
    class Variables:
        def weight_variable(shape):
            """
            Initialize weight matrices with truncated normal distribution with standard deviation of 0.1.

            In general, shape = [filter_height, filter_width, in_channels, out_channels].
            """
            stddev = 0.1
            size = 1
            for dim in shape:
                size *= dim
            return truncnorm.rvs(-2 * stddev, 2 * stddev, size=size).reshape(shape)

        def bias_variable(shape):
            """
            Initialize bias vectors with constant value of 0.1.

            In general, shape = [out_channels].
            """
            const = 0.1
            return np.full(shape, const)
    
    
    class Activations:
        class relu:
            def __init__(self, z):
                """Compute the rectified linear unit using z."""
                self.z = z
                self.mask_forward = (z >= 0).astype(np.float32)
                mask_zero = (z == 0).astype(np.float32)
                self.mask_backward = self.mask_forward - 0.5 * mask_zero

            def out(self):
                self.out = self.z * self.mask_forward
                return self.out

            def grad_z(self, grad_out):
                """Compute the derivative of the rectified linear unit using z."""
                print(grad_out)
                self.grad_z = grad_out * self.mask_backward
                return self.grad_z


        class softmax:
            def __init__(self, z):
                """Compute the softmax of z."""
                self.z = z

            def out(self):
                z_reshaped = self.z.reshape((self.z.shape[0], -1))
                z_max_reshaped = z_reshaped.max(axis = 1, keepdims = True)
                e_z_reshaped = np.exp(z_reshaped - z_max_reshaped)
                e_z_sum_reshaped = e_z_reshaped.sum(axis = 1, keepdims = True)
                out_reshaped = e_z_reshaped / e_z_sum_reshaped
                self.out = out_reshaped.reshape(self.z.shape)
                return self.out
            
            def grad_z(self, grad_out):
                """Compute the derivative of the softmax of z."""
                # self.grad_z = (grad_out * self.out) - grad_out.dot(np.outer(self.out, self.out))
                grad_out_reshaped = grad_out.reshape((grad_out.shape[0], -1))
                out_reshaped = self.out.reshape((self.out.shape[0], -1))
                inner = np.diagonal(grad_out_reshaped.dot(out_reshaped.T)).reshape((-1, 1))
                self.grad_z = (grad_out_reshaped - inner).reshape(grad_out.shape) * self.out
                return self.grad_z
    
    
    class Layers:
        class flatten:
            def __init__(self, x):
                """Flatten tensor to shape [batch_size, x_height * x_width * in_channels]."""
                self.x = x

            def out(self):
                self.out_val = self.x.reshape((self.x.shape[0], -1))
                return self.out_val
            
            def grad_x(self, grad_out):
                self.grad_x_val = grad_out.reshape(self.x.shape)
                return self.grad_x_val
            
        
        class fullconn:
            def __init__(self, x, w):
                """Fully connected layer."""
                assert(x.shape[-1] == w.shape[0])
                self.x = x
                self.w = w
                self.batch_size = x.shape[0]
                self.in_channels = x.shape[1]
                self.out_channels = w.shape[1]

            def out(self):
                self.out_val = self.x.dot(self.w)
                return self.out_val

            def grad_x(self, grad_out):
                self.grad_x_val = grad_out.dot(self.w.T)
                return self.grad_x_val

            def grad_w(self, grad_out):
                self.grad_w_val = self.x.T.dot(grad_out)
                return self.grad_w_val
            
            
        class conv2d:
            def __init__(self, x, w, strides=[1, 1, 1, 1], padding='SAME'):
                assert(x.shape[3] == w.shape[2])
                self.x = x
                self.w = w
                self.strides = strides
                self.batch_size = x.shape[0]
                self.in_height = x.shape[1]
                self.in_width = x.shape[2]
                self.filter_height = w.shape[0]
                self.filter_width = w.shape[1]
                self.in_channels = w.shape[2]
                self.out_channels = w.shape[3]
                
                ksize = (1, self.filter_height, self.filter_width, 1)
                padding_out = NN.Utils.pad(x, ksize, strides, padding = padding)
                self.pad_top = padding_out[0]
                self.pad_bottom = padding_out[1]
                self.pad_left = padding_out[2]
                self.pad_right = padding_out[3]
                self.out_height = padding_out[4]
                self.out_width = padding_out[5]
                self.x_pad = np.pad(x, [(0, 0), (self.pad_top, self.pad_bottom), (self.pad_left, self.pad_right), (0, 0)], mode='constant')
            
            def out_bkp(self):
                self.out = np.empty((self.batch_size, self.out_height, self.out_width, self.out_channels))
                for b in range(self.batch_size):
                    for i in range(self.out_height):
                        for j in range(self.out_width):
                            for k in range(self.out_channels):
                                self.out[b, i, j, k] = (self.x_pad[b, (self.strides[1] * i):(self.strides[1] * i + self.filter_height), (self.strides[2] * j):(self.strides[2] * j + self.filter_width), :] * self.w[::-1, ::-1, :, k]).sum(axis = (0, 1))
                return self.out
            
            def out(self):
                # x_col = NN.Utils.img_to_col(self.x_pad, self.out_height, self.out_width, self.filter_height, self.filter_width, self.strides)
                x_mat = img_to_mat(self.x_pad, self.out_height, self.out_width, self.filter_height, self.filter_width, self.strides)
#                 w_mat = self.w[::-1, ::-1].reshape((-1, self.out_channels))
                w_mat = self.w.reshape((-1, self.out_channels))
                self.fullconn = NN.Layers.fullconn(x_mat, w_mat)
                out_mat = self.fullconn.out()
                self.out_val = out_mat.reshape((self.batch_size, self.out_height, self.out_width, self.out_channels))
                return self.out_val
            
            def grad_x(self, grad_out):
                grad_out_reshaped = grad_out.reshape((-1, self.out_channels))
                grad_x_mat = self.fullconn.grad_x(grad_out_reshaped)
                # self.grad_x_val = NN.Utils.col_to_img(grad_x_col, self.out_height, self.out_width, self.x_pad.shape[1], self.x_pad.shape[2], self.filter_height, self.filter_width, self.strides)
                self.grad_x_val = mat_to_img(grad_x_mat, self.out_height, self.out_width, self.x_pad.shape[1], self.x_pad.shape[2], self.filter_height, self.filter_width, self.strides)
                if self.pad_top > 0:
                    self.grad_x_val = self.grad_x_val[:, self.pad_top:, :, :]
                if self.pad_bottom > 0:
                    self.grad_x_val = self.grad_x_val[:, :-self.pad_bottom, :, :]
                if self.pad_left > 0:
                    self.grad_x_val = self.grad_x_val[:, :, self.pad_left:, :]
                if self.pad_right > 0:
                    self.grad_x_val = self.grad_x_val[:, :, :-self.pad_right, :]
                return self.grad_x_val

            def grad_w(self, grad_out):
                grad_out_reshaped = grad_out.reshape((self.batch_size * self.out_height * self.out_width, self.out_channels))
                grad_w_col = self.fullconn.grad_w(grad_out_reshaped)
                self.grad_w_val = grad_w_col.reshape(self.w.shape)
                return self.grad_w_val


        class maxpool2d:
            def __init__(self, x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME'):
                self.x = x
                self.ksize = ksize
                self.strides = strides
                self.batch_size = x.shape[0]
                self.in_height = x.shape[1]
                self.in_width = x.shape[2]
                self.in_channels = x.shape[3]
                self.out_channels = self.in_channels
                padding_out = NN.Utils.pad(x, ksize, strides, padding = padding)
                self.pad_top = padding_out[0]
                self.pad_bottom = padding_out[1]
                self.pad_left = padding_out[2]
                self.pad_right = padding_out[3]
                self.out_height = padding_out[4]
                self.out_width = padding_out[5]
                self.x_pad = np.pad(x, [(0, 0), (self.pad_top, self.pad_bottom), (self.pad_left, self.pad_right), (0, 0)], mode='constant')

            def out_bkp(self):
                self.out_val = np.empty((self.batch_size, self.out_height, self.out_width, self.out_channels))
                self.pos = np.zeros((self.batch_size, self.out_height, self.out_width, self.out_channels, 4), dtype = int)
                for b in range(self.batch_size):
                    for i in range(self.out_height):
                        stride_i = self.strides[1] * i
                        for j in range(self.out_width):
                            stride_j = self.strides[2] * j
                            for k in range(self.out_channels):
                                x_block = self.x_pad[b, (self.strides[1] * i):(self.strides[1] * i + self.ksize[1]), (self.strides[2] * j):(self.strides[2] * j + self.ksize[2]), k]
                                max_val = x_block.max()
                                self.out_val[b, i, j, k] = max_val
                                max_inds = list(np.unravel_index(x_block.argmax(), x_block.shape))
                                self.pos[b, i, j, k] = np.array([b, self.strides[1] * i + max_inds[0], self.strides[2] * j + max_inds[1], k])
                return self.out_val
            
            def grad_x_bkp(self, grad_out):
                self.grad_x_val = np.zeros(self.x_pad.shape)
                for b in range(self.batch_size):
                    for i in range(self.out_height):
                        for j in range(self.out_width):
                            for k in range(self.out_channels):
                                self.grad_x_val[tuple(self.pos[b, i, j, k])] = grad_out[b, i, j, k]
                return self.grad_x_val
            
            def out(self):
                x_reshaped = self.x_pad.transpose((3, 0, 1, 2)).reshape((self.in_channels * self.batch_size, self.in_height, self.in_width, 1))
                # x_col = NN.Utils.img_to_col(x_reshaped, self.out_height, self.out_width, self.ksize[1], self.ksize[2], self.strides)
                x_mat = img_to_mat(x_reshaped, self.out_height, self.out_width, self.ksize[1], self.ksize[2], self.strides)
                self.x_mat_maxcols = x_mat.argmax(axis = 1)
                out_reshaped = x_mat[np.arange(x_mat.shape[0]), self.x_mat_maxcols]
                self.out_val = out_reshaped.reshape((self.in_channels, self.batch_size, self.out_height, self.out_width)).transpose((1, 2, 3, 0))
                return self.out_val
            
            def grad_x(self, grad_out):
                grad_x_mat = np.zeros((self.in_channels * self.batch_size * self.out_height * self.out_width, self.ksize[1] * self.ksize[2]))
                grad_x_mat[np.arange(grad_x_mat.shape[0]), self.x_mat_maxcols] = grad_out.transpose((3, 0, 1, 2)).flatten()
                # grad_x_reshaped = NN.Utils.col_to_img(grad_x_mat, self.out_height, self.out_width, self.x_pad.shape[1], self.x_pad.shape[2], self.ksize[1], self.ksize[2], self.strides)
                grad_x_reshaped = mat_to_img(grad_x_mat, self.out_height, self.out_width, self.x_pad.shape[1], self.x_pad.shape[2], self.ksize[1], self.ksize[2], self.strides)
                if self.pad_top > 0:
                    grad_x_reshaped = grad_x_reshaped[:, self.pad_top:, :, :]
                if self.pad_bottom > 0:
                    grad_x_reshaped = grad_x_reshaped[:, :-self.pad_bottom, :, :]
                if self.pad_left > 0:
                    grad_x_reshaped = grad_x_reshaped[:, :, self.pad_left:, :]
                if self.pad_right > 0:
                    grad_x_reshaped = grad_x_reshaped[:, :, :-self.pad_right, :]
                self.grad_x_val = grad_x_reshaped.reshape((self.in_channels, self.batch_size, self.in_height, self.in_width)).transpose((1, 2, 3, 0))
                return self.grad_x_val
    
    
    class Cost:
        class mean_squared_error:
            def __init__(self, y_label, y_out):
                self.y_label = y_label
                self.y_out = y_out
            
            def out(self):
                self.out_val = 0.5 * (self.y_label - self.y_out) ** 2
                return self.out_val
            
            def grad_x(self):
                self.grad_x_val = self.y_out - self.y_label
                return self.grad_x_val


        class cross_entropy:
            def __init__(self, y_label, y_out):
                self.y_label = y_label
                self.y_out = y_out
            
            def out(self):
                self.out_val = -np.sum(self.y_label * np.log(self.y_out), axis = 1)
                return self.out_val
            
            def grad_x(self):
                self.grad_x_val = -self.y_label / self.y_out
                return self.grad_x_val


        class softmax_cross_entropy:
            def __init__(self, y_label, y_out):
                self.y_label = y_label
                self.y_out = y_out
            
            def out(self):
                softmax = NN.Activations.softmax(self.y_out)
                self.out_val = -np.sum(self.y_label * np.log(softmax.out()), axis = 1)
                return self.out_val
            
            def grad_z(self):
                self.grad_z_val = self.y_out - self.y_label
                return self.grad_z_val
    
    
    class Metrics:
        def accuracy(y_label, y_out):
            correct_prediction = np.equal(np.argmax(y_label, axis = 1), np.argmax(y_out, axis = 1))
            return correct_prediction.astype(np.float32).mean()


    class Optimizers:
        class Adagrad:
            def __init__(self, lr, n_layers, initial_accumulator_value=0.1):
                self.lr = lr
                self.dw_sum_sq = [initial_accumulator_value for i in range(n_layers)]
                self.db_sum_sq = [initial_accumulator_value for i in range(n_layers)]

            def step(self, dw, db):
                for i in range(len(dw)):
                    if not dw[i] is None:
                        self.dw_sum_sq[i] += dw[i] ** 2
                    if not db[i] is None:
                        self.db_sum_sq[i] += db[i] ** 2

            def lr_eff(self):
                lr_eff_w = []
                lr_eff_b = []
                for i in range(len(self.dw_sum_sq)):
                    # print((self.lr, np.sqrt(self.dw_sum_sq[i]).shape, self.lr / np.sqrt(self.dw_sum_sq[i])))
                    lr_eff_w.append(self.lr / np.sqrt(self.dw_sum_sq[i]))
                    lr_eff_b.append(self.lr / np.sqrt(self.db_sum_sq[i]))
                return lr_eff_w, lr_eff_b

In [4]:
class Model:
    def __init__(self, x_image_shape=[28, 28], lr=1e-3, optimizer=None, cost=NN.Cost.mean_squared_error):
        self.started = False
        self.epochs_completed = 0
        self.x_image_shape = x_image_shape
        self.lr = lr
        self.optimizer_func = optimizer
        self.cost_func = cost
        self.w = []
        self.dw = []
        self.b = []
        self.db = []
        self.f = []
        self.s = []

    def optimize_with(self, optimizer_func, lr=None):
        self.optimizer_func = optimizer_func
        if lr != self.lr:
            self.lr = lr
        return self

    def minimize(self, cost_func):
        self.cost_func = cost_func
        return self

    def init_layer_vars(self, w=None, b=None):
        if not self.started:
            self.w.append(w)
            self.dw.append(None)
            self.b.append(b)
            self.db.append(None)
            self.f.append(None)
            self.s.append(None)
        
    def model(self, batch_x_images):
        # Layer 0 (2D convolution layer)
        filter_shape_0 = [10, 10]
        in_channels_0 = 1
        out_channels_0 = 32

        # Reshape input
        x_0 = np.reshape(batch_x_images, [batch_x_images.shape[0]] + list(self.x_image_shape) + [in_channels_0])

        self.init_layer_vars(w = NN.Variables.weight_variable(filter_shape_0 + [in_channels_0, out_channels_0]),
                             b = NN.Variables.bias_variable([out_channels_0]))

        self.f[0] = NN.Layers.conv2d(x_0, self.w[0])
        z_0 = self.f[0].out() + self.b[0]
        
        self.s[0] = NN.Activations.relu(z_0)
        x_1 = self.s[0].out()

        # Layer 1 (Max pool 2x2)
        in_channels_1 = out_channels_0
        out_channels_1 = in_channels_1
        
        self.init_layer_vars()

        self.f[1] = NN.Layers.maxpool2d(x_1)
        x_2 = self.f[1].out()

        # Layer 2 (2D convolution layer)
        filter_shape_2 = [5, 5]
        in_channels_2 = out_channels_1
        out_channels_2 = 16

        self.init_layer_vars(w = NN.Variables.weight_variable(filter_shape_2 + [in_channels_2, out_channels_2]),
                             b = NN.Variables.bias_variable([out_channels_2]))

        self.f[2] = NN.Layers.conv2d(x_2, self.w[2])
        z_2 = self.f[2].out() + self.b[2]
        
        self.s[2] = NN.Activations.relu(z_2)
        x_3 = self.s[2].out()

        # Layer 3 (Max pool 2x2)
        in_channels_3 = out_channels_2
        out_channels_3 = in_channels_3
        
        self.init_layer_vars()

        self.f[3] = NN.Layers.maxpool2d(x_3)
        x_4 = self.f[3].out()

        # Layer 4 (Flatten)
        in_channels_4 = out_channels_3
        out_channels_4 = in_channels_4
        
        self.init_layer_vars()

        self.f[4] = NN.Layers.flatten(x_4)
        x_5 = self.f[4].out()

        # Layer 5 (Fully connected layer)
        in_channels_5 = x_5.shape[-1]
        out_channels_5 = 1024

        self.init_layer_vars(w = NN.Variables.weight_variable([in_channels_5, out_channels_5]),
                             b = NN.Variables.bias_variable([out_channels_5]))

        self.f[5] = NN.Layers.fullconn(x_5, self.w[5])
        z_5 = self.f[5].out() + self.b[5]

        self.s[5] = NN.Activations.relu(z_5)
        x_6 = self.s[5].out()

        # Layer 6 (Fully connected layer)
        in_channels_6 = out_channels_5
        out_channels_6 = 10

        self.init_layer_vars(w = NN.Variables.weight_variable([in_channels_6, out_channels_6]),
                             b = NN.Variables.bias_variable([out_channels_6]))

        self.f[6] = NN.Layers.fullconn(x_6, self.w[6])
        z_6 = self.f[6].out() + self.b[6]

        self.s[6] = NN.Activations.softmax(z_6)
        x_7 = self.s[6].out()
        
        self.batch_y_out = x_7
        
        return self.batch_y_out
    
    def backpropagate(self, batch_y_labels):
        n_layers = len(self.dw)
        if not self.started:
            if not self.optimizer_func is None:
                self.optimizer = self.optimizer_func(self.lr, n_layers)
            self.started = True

        cost = self.cost_func(batch_y_labels, self.batch_y_out)

        if self.optimizer_func is None:
            lr_eff_w, lr_eff_b = [self.lr for i in range(n_layers)], [self.lr for i in range(n_layers)]
        else:
            lr_eff_w, lr_eff_b = self.optimizer.lr_eff()

        if 'grad_z' in vars(self.cost_func):
            delta = -cost.grad_z()
        elif 'grad_x' in vars(self.cost_func):
            delta = -self.s[n_layers - 1].grad_z(cost.grad_x())
        else:
            raise Exception("No cost function recognized.")

        if not self.w[n_layers - 1] is None:
            # print(np.mean(np.sum(delta, axis = tuple(range(1, delta.ndim - 1))), axis = 0))
            self.db[n_layers - 1] = lr_eff_b[n_layers - 1] * np.sum(delta, axis = tuple(range(delta.ndim - 1)))
            self.b[n_layers - 1] += self.db[n_layers - 1]
            # print(self.f[n_layers - 1].grad_w(delta).shape)
            self.dw[n_layers - 1] = lr_eff_w[n_layers - 1] * self.f[n_layers - 1].grad_w(delta)
            # print("{}: {}".format(n_layers - 1, self.dw[n_layers - 1]))
            self.w[n_layers - 1] += self.dw[n_layers - 1]
        
        for i in reversed(range(n_layers - 1)):
            if self.w[i] is None:
                delta = self.f[i + 1].grad_x(delta)
            else:
                delta = self.s[i].grad_z(self.f[i + 1].grad_x(delta))
                # print(delta.shape)
                self.db[i] = lr_eff_b[i] * np.sum(delta, axis = tuple(range(delta.ndim - 1)))
                self.b[i] += self.db[i]
                # print(self.f[i].grad_w(delta).shape)
                self.dw[i] = lr_eff_w[i] * self.f[i].grad_w(delta)
                # print("{}: {}".format(i, self.dw[i]))
                self.w[i] += self.dw[i]

        if not self.optimizer_func is None:
            self.optimizer.step(self.dw, self.db)

    def train_step(self, batch_x_images, batch_y_labels, timing=False):
        batch_size = batch_x_images.shape[0]
        if timing:
            start_forward_time = time()
        batch_y_out = self.model(batch_x_images)
        if timing:
            forward_time = (time() - start_forward_time) / batch_size
            start_backward_time = time()
        self.backpropagate(batch_y_labels)
        if timing:
            backward_time = (time() - start_backward_time) / batch_size
            return batch_y_out, forward_time, backward_time
        return batch_y_out

In [5]:
# Import data
path = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(path, one_hot=True)

batch_x_images, batch_y_labels = mnist.train.next_batch(5)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


In [6]:
sess = tf.InteractiveSession()
model = Model(lr = 1e-3).minimize(NN.Cost.softmax_cross_entropy)

In [7]:
# Part 2
x_image_shape = [28, 28]
x = tf.placeholder(tf.float32, [None, 784])

# Layer 0 (2D convolution layer)
filter_shape_0 = [10, 10]
in_channels_0 = 1
out_channels_0 = 32

# Reshape input
batch = -1
tf_x_0 = tf.reshape(x, [batch] + list(x_image_shape) + [in_channels_0])

tf_w_0 = weight_variable(filter_shape_0 + [in_channels_0, out_channels_0])
sess.run(tf_w_0.initializer)
tf_b_0 = bias_variable([out_channels_0])
sess.run(tf_b_0.initializer)

tf_z_0 = conv2d(tf_x_0, tf_w_0) + tf_b_0

tf_x_1 = tf.nn.relu(tf_z_0)

# Layer 1 (Max pool 2x2)
in_channels_1 = out_channels_0
out_channels_1 = in_channels_1
    
tf_x_2 = maxpool2d(tf_x_1)

# Layer 2 (2D convolution layer)
filter_shape_2 = [5, 5]
in_channels_2 = out_channels_1
out_channels_2 = 16

tf_w_2 = weight_variable(filter_shape_2 + [in_channels_2, out_channels_2])
sess.run(tf_w_2.initializer)
tf_b_2 = bias_variable([out_channels_2])
sess.run(tf_b_2.initializer)

tf_z_2 = conv2d(tf_x_2, tf_w_2) + tf_b_2

tf_x_3 = tf.nn.relu(tf_z_2)

# Layer 3 (Max pool 2x2)
in_channels_3 = out_channels_2
out_channels_3 = in_channels_3
    
tf_x_4 = maxpool2d(tf_x_3)

# Layer 4 (Flatten)
in_channels_4 = out_channels_3
out_channels_4 = in_channels_4

tf_x_5 = flatten(tf_x_4)

# Layer 5 (Fully connected layer)
in_channels_5 = tf_x_5.shape.as_list()[-1]
out_channels_5 = 1024

tf_w_5 = weight_variable([in_channels_5, out_channels_5])
sess.run(tf_w_5.initializer)
tf_b_5 = bias_variable([out_channels_5])
sess.run(tf_b_5.initializer)

tf_z_5 = fullconn(tf_x_5, tf_w_5) + tf_b_5

tf_x_6 = tf.nn.relu(tf_z_5)

# Layer 6 (Fully connected layer)
in_channels_6 = out_channels_5
out_channels_6 = 10

tf_w_6 = weight_variable([in_channels_6, out_channels_6])
sess.run(tf_w_6.initializer)
tf_b_6 = bias_variable([out_channels_6])
sess.run(tf_b_6.initializer)

tf_z_6 = fullconn(tf_x_6, tf_w_6) + tf_b_6

y = tf_z_6
y_ = tf.placeholder(tf.float32, [None, 10])

In [8]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdagradOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

In [9]:
# Part 3
# Layer 0 (2D convolution layer)
filter_shape_0 = [10, 10]
in_channels_0 = 1
out_channels_0 = 32

# Reshape input
x_0 = np.reshape(batch_x_images, [batch_x_images.shape[0]] + list(model.x_image_shape) + [in_channels_0])

model.init_layer_vars(w = sess.run(tf_w_0),
                     b = sess.run(tf_b_0))

model.f[0] = NN.Layers.conv2d(x_0, model.w[0])
z_0 = model.f[0].out() + model.b[0]

model.s[0] = NN.Activations.relu(z_0)
x_1 = model.s[0].out()

# Layer 1 (Max pool 2x2)
in_channels_1 = out_channels_0
out_channels_1 = in_channels_1

model.init_layer_vars()

model.f[1] = NN.Layers.maxpool2d(x_1)
x_2 = model.f[1].out()

# Layer 2 (2D convolution layer)
filter_shape_2 = [5, 5]
in_channels_2 = out_channels_1
out_channels_2 = 16

model.init_layer_vars(w = sess.run(tf_w_2),
                     b = sess.run(tf_b_2))

model.f[2] = NN.Layers.conv2d(x_2, model.w[2])
z_2 = model.f[2].out() + model.b[2]

model.s[2] = NN.Activations.relu(z_2)
x_3 = model.s[2].out()

# Layer 3 (Max pool 2x2)
in_channels_3 = out_channels_2
out_channels_3 = in_channels_3

model.init_layer_vars()

model.f[3] = NN.Layers.maxpool2d(x_3)
x_4 = model.f[3].out()

# Layer 4 (Flatten)
in_channels_4 = out_channels_3
out_channels_4 = in_channels_4

model.init_layer_vars()

model.f[4] = NN.Layers.flatten(x_4)
x_5 = model.f[4].out()

# Layer 5 (Fully connected layer)
in_channels_5 = x_5.shape[-1]
out_channels_5 = 1024

model.init_layer_vars(w = sess.run(tf_w_5),
                     b = sess.run(tf_b_5))

model.f[5] = NN.Layers.fullconn(x_5, model.w[5])
z_5 = model.f[5].out() + model.b[5]

model.s[5] = NN.Activations.relu(z_5)
x_6 = model.s[5].out()

# Layer 6 (Fully connected layer)
in_channels_6 = out_channels_5
out_channels_6 = 10

model.init_layer_vars(w = sess.run(tf_w_6),
                     b = sess.run(tf_b_6))

model.f[6] = NN.Layers.fullconn(x_6, model.w[6])
z_6 = model.f[6].out() + model.b[6]

model.s[6] = NN.Activations.softmax(z_6)
x_7 = model.s[6].out()

model.batch_y_out = x_7

model.started = True

In [47]:
np.allclose(model.w[0], sess.run(tf_w_0))

False

In [41]:
(x_7 - sess.run(tf.nn.softmax(tf_z_6), feed_dict={x: batch_x_images, y_: batch_y_labels})).max(), x_7.mean()

(9.8347664e-07, 0.1)

In [14]:
sess.run(tf_z_0, feed_dict={x: batch_x_images, y_: batch_y_labels})[0, :, :, 0]

array([[  1.00000001e-01,   1.00000001e-01,   1.00000001e-01,
          1.00000001e-01,   1.00000001e-01,   1.00000001e-01,
          1.00000001e-01,   1.00000001e-01,   1.07168444e-01,
          1.24769822e-01,   1.71549857e-01,   2.65877604e-01,
          3.21641982e-01,   3.85279626e-01,   3.76678199e-01,
          2.55690515e-01,   2.10916668e-01,   1.84303433e-01,
         -9.55344662e-02,  -2.13925153e-01,  -9.28050503e-02,
         -1.93637982e-02,  -2.47329846e-02,  -3.67036089e-02,
          7.17024803e-02,   1.00000001e-01,   1.00000001e-01,
          1.00000001e-01],
       [  1.00000001e-01,   1.00000001e-01,   1.00000001e-01,
          1.00000001e-01,   1.00000001e-01,   1.00000001e-01,
          1.00000001e-01,   1.19937226e-01,   1.91691026e-01,
          2.49636263e-01,   3.30932796e-01,   3.54055703e-01,
          2.44973123e-01,   2.13747621e-01,   2.38689244e-01,
          1.75958440e-01,   2.21730098e-02,   6.51277602e-03,
          9.32156146e-02,  -6.11844733e-02,

In [42]:
sess.run(train_step, feed_dict={x: batch_x_images, y_: batch_y_labels})

In [45]:
model.backpropagate(batch_y_labels)

[[-0.04295551  0.03543454  0.01056092 ...,  0.04396503 -0.08325317
   0.00572435]
 [ 0.03448073  0.23536521  0.07970923 ..., -0.20097183 -0.14345494
  -0.07462213]
 [ 0.07669418  0.20575476 -0.02404383 ...,  0.04536538 -0.10266566
  -0.05931497]
 [ 0.09231361  0.06843063 -0.13852426 ..., -0.03225738 -0.02552938
   0.03710412]
 [-0.03323843  0.09567886  0.07424027 ..., -0.10778412 -0.17537043
  -0.02384581]]
[[[[  0.00000000e+00   0.00000000e+00   5.75848783e-01 ...,
      0.00000000e+00   0.00000000e+00   6.50514220e-02]
   [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,
      0.00000000e+00   0.00000000e+00   0.00000000e+00]
   [  0.00000000e+00   0.00000000e+00   3.22158461e-01 ...,
      0.00000000e+00   0.00000000e+00   1.95586717e-02]
   ..., 
   [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,
      0.00000000e+00   0.00000000e+00   0.00000000e+00]
   [  0.00000000e+00   0.00000000e+00   1.80629732e-01 ...,
      0.00000000e+00  -2.36612352e-02   1.67177226e-01]


[[[[  3.31044200e-02  -1.36255598e-01  -6.74251433e-02 ...,
     -9.48703338e-02  -1.39567677e-01  -8.91212284e-02]
   [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,
      0.00000000e+00   0.00000000e+00   0.00000000e+00]
   [  8.22907173e-02   5.77564678e-02   4.69702985e-02 ...,
     -1.45529601e-01  -6.77193997e-02   3.24047003e-02]
   ..., 
   [ -3.58800201e-02   0.00000000e+00   0.00000000e+00 ...,
     -7.88139352e-02  -6.15090063e-02   0.00000000e+00]
   [  5.59884140e-03   1.76273498e-01  -5.40160978e-03 ...,
      2.65021498e-02   2.11829394e-02   4.94069636e-02]
   [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,
      0.00000000e+00   0.00000000e+00   0.00000000e+00]]

  [[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,
      0.00000000e+00   0.00000000e+00   0.00000000e+00]
   [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,
      0.00000000e+00   0.00000000e+00   0.00000000e+00]
   [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,
 

In [54]:
(model.w[6] - sess.run(tf_w_6)).max(), model.w[6].mean()

(0.0044566728, 0.00029872436)

In [61]:
sess.run(tf_w_6)

array([[-0.04658139,  0.07002026, -0.02467365, ..., -0.01310672,
         0.03694488,  0.03129107],
       [-0.0423685 ,  0.14428018, -0.0073563 , ..., -0.08457146,
        -0.18001738,  0.1348218 ],
       [-0.00249426, -0.013936  ,  0.06616337, ...,  0.02536744,
        -0.09356379,  0.0755139 ],
       ..., 
       [ 0.08165683,  0.08867195, -0.05271607, ...,  0.0578579 ,
         0.05865176, -0.14618491],
       [ 0.02221281,  0.0317058 , -0.0414342 , ...,  0.17885755,
         0.08150197,  0.00619883],
       [-0.02146946, -0.10103993, -0.0563484 , ..., -0.05636124,
         0.02116269, -0.11503019]], dtype=float32)

In [62]:
model.w[6]

array([[-0.04647477,  0.07034583, -0.02416823, ..., -0.01498944,
         0.03658298,  0.03178586],
       [-0.04144545,  0.14449441, -0.00711204, ..., -0.08631093,
        -0.18046908,  0.13500033],
       [-0.00133603, -0.0137419 ,  0.06715759, ...,  0.02082238,
        -0.0946267 ,  0.07627748],
       ..., 
       [ 0.08212607,  0.08902931, -0.05192495, ...,  0.05667943,
         0.05825147, -0.14618511],
       [ 0.02296838,  0.03170336, -0.04134893, ...,  0.17702755,
         0.08111281,  0.00654483],
       [-0.02068534, -0.10032496, -0.05563926, ..., -0.0586389 ,
         0.02049781, -0.1144373 ]], dtype=float32)